In [ ]:
# Import & Load Scan
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from src.preprocessing import normalize, resample

scan_path = "data/sample_patient.nii.gz"
scan = nib.load(scan_path).get_fdata()

# Normalize
scan_norm = normalize(scan)

# View middle slice
plt.imshow(scan_norm[scan_norm.shape[0] // 2], cmap='gray')
plt.title("Mid Slice")
plt.axis('off')
plt.show()
#Visualize 3D
def plot_3d(volume, threshold=-300):
    p = volume.transpose(2,1,0)
    from skimage import measure
    verts, faces, _, _ = measure.marching_cubes(p, level=threshold)
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces], alpha=0.7)
    mesh.set_facecolor('red')
    ax.add_collection3d(mesh)
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    plt.show()

plot_3d(scan_norm)
#Grad-CAM Heatmap
from src.explain import generate_gradcam
from torchvision import models, transforms
from PIL import Image
import torch

model = torch.load("../models/densenet_model.pth")
model.eval()

image = Image.open("sample_image.jpg").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
tensor = transform(image).unsqueeze(0)

# Target class (e.g., 1 for malignant)
cam_image = generate_gradcam(model, tensor, 1, model.features[-1], np.array(image)/255.0)

plt.imshow(cam_image)
plt.title("Grad-CAM")
plt.axis('off')
plt.show()
#SHAP Explanation
import shap
import numpy as np

# DeepExplainer requires model input and reference/background
background = tensor.repeat(10, 1, 1, 1)  # dummy background
explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(tensor)

shap.image_plot(shap_values, np.array([np.array(image) / 255.0]))



